In [1]:
# !pip install docx2txt
# !pip install sentence_transformers
# !pip install PyPDF2
# !pip install hashlib

In [36]:
#Importing the required packages

import os
import hashlib
from shutil import move

from sentence_transformers import SentenceTransformer
import PyPDF2
import docx2txt

import pickle

## SET PATH

In [3]:
# Define directory paths
document_folder = r"C:\Users\ankit\Downloads\Duplicate files\Files"
duplicates_folder = r"C:\Users\ankit\Downloads\Duplicate files\Files\Duplicate_files"

#Define similarity score
cosine_score=0.9

###  Defining HASH and SHA Algorithm

In [16]:
# Defining the SHA Algorithm
def create_hash_database(folder_path):
    hash_database = {} #creating a dictionary for hash_database
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                with open(file_path, 'rb') as f:
                    md5_hash = hashlib.md5(f.read()).hexdigest()
                    sha256_hash = hashlib.sha256(f.read()).hexdigest()
                hash_database[filename] = (md5_hash, sha256_hash)
            except FileNotFoundError:
                print(f"Error: File '{filename}' not found. Hence skipping...")
    return hash_database

def create_embd_database(folder_path, destination_folder):
    embd_database = {} #creating a dictionary for embd_database
    print("wfeew")
    print(os.listdir(folder_path))
    for filename in os.listdir(folder_path):
        print(filename)
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            print("inside row 21",filename)
            try:
                with open(file_path, 'rb') as f:
                    embedding=model.encode(document_folder, normalize_embeddings=True)
                    print(filename,"rfwer",embedding)
                # Moving duplicates in intial database creation if cosine similarity is greter than cosine score
                for key,value in embd_database.items():
                    similarity_score=value @ embedding.T
                    print(similarity_score,key,value) ##
                    if similarity_score> cosine_score:
                        move_duplicate_llm(filename,value,similarity_score,folder_path, destination_folder)
                else:
                    embd_database[filename] = embedding
            except FileNotFoundError:
                print(f"Error: File '{filename}' not found. Hence skipping...")
            except Exception as e:
                print(f"Error moving '{filename}': {e}")
    return embd_database

def check_duplicate(filename, hash_database):
    if filename in hash_database:
        return True
    return False

def move_duplicate_llm(filename,duplicate_file,similarity_score, source_folder, destination_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    try:
        move(source_path, destination_path)
        print(f"Duplicate '{filename}' moved to duplicate folder with similarity score of {similarity_score} with file {duplicate_file}")
    except Exception as e:
        print(f"Error moving '{filename}': {e}")
        

def move_duplicate(filename, source_folder, destination_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    try:
        move(source_path, destination_path)
        print(f"Duplicate '{filename}' moved to duplicate folder.")
    except Exception as e:
        print(f"Error moving '{filename}': {e}")

def extract_text(path):
    text = ""
    file_ext= path.split('.')[-1]    
    if file_ext == 'docx':
        text =docx2txt.process(path)
        
    elif file_ext == 'pdf':
        with open(path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
    else:
        pass

    return text

In [43]:
if __name__ == "__main__":
    model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
    #sha_algo()
    LLM_algo()

LLM model is running....
Creating Database
wfeew
['Duplicate_files', 'Hugging Face - Copy.docx', 'Hugging Face.docx', 'Hugging Face.pdf', 'Hugging Face_V2.pdf', 'Kupdf_net_irctc_ticket_format - Copy.pdf', 'Kupdf_net_irctc_ticket_format.pdf', 'Machine learning - Wikipedia_1page.pdf', 'MicrosoftTeams-image (1).png', 'MicrosoftTeams-image (2) - Copy.png', 'MicrosoftTeams-image (2).png', 'MicrosoftTeams-image (3).png', 'MicrosoftTeams-image - Copy.png', 'MicrosoftTeams-image.png', 'sample.doc']
Duplicate_files
Hugging Face - Copy.docx
inside row 21 Hugging Face - Copy.docx
Hugging Face - Copy.docx rfwer [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Hugging Face.docx
inside row 21 Hugging Face.docx
Hugging Face.docx rfwer [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
1.0000001 Hugging Face - Copy.docx [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Duplicate 'Hugging Face.docx' moved to duplicat

MicrosoftTeams-image (2) - Copy.png rfwer [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
1.0000001 Hugging Face - Copy.docx [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Duplicate 'MicrosoftTeams-image (2) - Copy.png' moved to duplicate folder with similarity score of 1.0000001192092896 with file [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
1.0000001 Hugging Face.docx [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Error moving 'MicrosoftTeams-image (2) - Copy.png': [Errno 2] No such file or directory: 'C:\\Users\\ankit\\Downloads\\Duplicate files\\Files\\MicrosoftTeams-image (2) - Copy.png'
1.0000001 Hugging Face.pdf [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Error moving 'MicrosoftTeams-image (2) - Copy.png': [Errno 2] No such file or directory: 'C:\\Users\\ankit\\Downloads\\Duplicate files\\Files\\MicrosoftTeams-image (

MicrosoftTeams-image - Copy.png rfwer [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
1.0000001 Hugging Face - Copy.docx [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Duplicate 'MicrosoftTeams-image - Copy.png' moved to duplicate folder with similarity score of 1.0000001192092896 with file [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
1.0000001 Hugging Face.docx [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Error moving 'MicrosoftTeams-image - Copy.png': [Errno 2] No such file or directory: 'C:\\Users\\ankit\\Downloads\\Duplicate files\\Files\\MicrosoftTeams-image - Copy.png'
1.0000001 Hugging Face.pdf [ 0.02250169  0.02356131 -0.01475737 ...  0.02637642  0.03610716
 -0.04403142]
Error moving 'MicrosoftTeams-image - Copy.png': [Errno 2] No such file or directory: 'C:\\Users\\ankit\\Downloads\\Duplicate files\\Files\\MicrosoftTeams-image - Copy.png'
1.0000001

Duplicate 'Hugging Face - Copy.docx' moved to duplicate folder with similarity score of 1.0000001192092896 with file Hugging Face.docx
Completed!


In [39]:
def LLM_algo():
    print('LLM model is running....')
    
    #Setting up the model bge-large-zh-v1.5
    
    
    if os.path.exists("embd_database.pkl"):
        with open("embd_database.pkl", "rb") as file:
            embd_database = pickle.load(file)
    else:
        print("Creating Database")
        embd_database = create_embd_database(document_folder,duplicates_folder)
        with open("embd_database.pkl", "wb") as file:
            pickle.dump(embd_database,file)
    
    for filename in os.listdir(document_folder):
        file_path = os.path.join(document_folder, filename)
        if os.path.isfile(file_path):
            #New document: update hash database and potentially move existing duplicates
            try:
                with open(file_path, "rb") as f:
                    embedding=model.encode(document_folder, normalize_embeddings=True)
                embd_database[filename] = embd_database
                
                #check for existing duplicates based on embedding
                for existing_filename, existing_embd in embd_database.items():
                    #checking for Duplicates
                    if(existing_filename != filename):
                        similarity_score= existing_embd @ embedding.T
                        if similarity_score> cosine_score: 
                            move_duplicate_llm(filename,existing_filename, similarity_score,document_folder, duplicates_folder)
                            break #only move one duplicate per new document
            except FileNotFoundError:
                print(f"Error: File'{filename}' not found. Skipping...")
    print("Completed!")
#         #Extracting text from Documents
#         sentences_1=extract_text(document_folder)
#         sentences_2=extract_text(duplicates_folder)

        

#         #Embedding the text of documents
#        
#         embeddings_2 = model.encode(duplicates_folder, normalize_embeddings=True)

#         #Claculating the similarity score
#         similarity = embeddings_1 @ embeddings_2.T

#         #Printing the statement
#         if similarity > 0.90:
#             print(f"Similar Document with similarity score of {similarity}")
#         else:
#             print(f"Different Document with similarity score of {similarity}")

In [44]:
z=[1,2,34,1]

In [5]:
def sha_algo():
    #load existing hash database or create a new one
    if os.path.exists("hash_database.txt"):
        with open("hash_database.txt", "r") as f:
            hash_database = eval(f.read())
    else:
        hash_database = create_hash_database(document_folder)
        with open("hash_database.txt", "w") as f:
            f.write(str(hash_database))
    
    #Check for new documents
    for filename in os.listdir(document_folder):
        file_path = os.path.join(document_folder, filename)
        if os.path.isfile(file_path):
            #New document: update hash database and potentially move existing duplicates
            try:
                with open(file_path, "rb") as f:
                    md5_hash = hashlib.md5(f.read()).hexdigest()
                    sha256_hash = hashlib.sha256(f.read()).hexdigest()
                hash_database[filename] = (md5_hash, sha256_hash)
                
                #check for existing duplicates based on MD5 or SHA-256
                for existing_filename, existing_hashes in hash_database.items():
                    if(existing_filename != filename) and (md5_hash == existing_hashes[0]): #checking for MD5
                        #move existing duplicates
                        move_duplicate(existing_filename, document_folder, duplicates_folder)
                        break #only move one duplicate per new document
            except FileNotFoundError:
                print(f"Error: File'{filename}' not found. Skipping...")

    with open("hash_database.txt", "w") as f:
        f.write(str(hash_database))
    print("Hash database updated.")
    

if __name__ == "__main__":
    #sha_algo()
    LLM_algo()